# Mô hình nhận diện từ vựng qua giọng nói bằng phương pháp phân loại và xác suất

- Mô hình phân loại là một hình rất phổ biến hiện nay, nhưng trong speech to text thì được rất ít dùng đến bởi vì phương pháp này có nhiều mặt hạn chế trong các câu và ngữ pháp, Nhưng phân biệt tự vựng nó mang lại được độ chính xác dễ triển khai mà không cần quá nhiều tài nguyên hơn seq2seq. Trong bài này chúng ta sẽ phân tích xây dựng một mô hình phân loại cơ bản thử nghiêm.

- Ưu điểm mô hình này là có thể giới hạn được số từ vựng trainning, và có thể trích xuất nhiều kết quả dự đoán theo dãy xác suất để người dùng nhận biết được nhầm lẫn trong các từ vựng.

In [9]:
import librosa
import numpy as np
import os
#lấy các thư mục dưới dạng tên của từ vựng
folder_path = 'data'
ds = os.listdir(folder_path)
#thu thập dự liệu âm thanh của từng labels
labels = []
data = []
for step in ds:
    files = os.listdir(f'data/{step}')
    for i in files:
        audio_data, sr = librosa.load(f'data/{step}/{i}', sr=None) 
        #lầm đều kích thước tranning ~2,5s
        audio_data = np.pad(audio_data, (0, 50000 - len(audio_data)), 'constant')
        #Trích xuất đặc trưng mfccs
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sr)
        data.append(mfccs)
        labels.append(step)  
        
data_s = np.array(data)
labels = np.array(labels)
print(data_s.shape)


(29940, 20, 98)


*Mã hóa labels sang dạng one-hot*

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
tokenizer = Tokenizer()
tokenizer.fit_on_texts(labels)
labelss = tokenizer.texts_to_sequences(labels)
#mã hóa one-hot
one_hot_labels = to_categorical(labelss)

*Chia tập dữ liệu huấn luận*

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_s, one_hot_labels, test_size=0.2, random_state=42)


**Huấn luyện với model NLP cơ bản**

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional

model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(20, 98)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dense(999, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train,validation_data=(X_test, y_test), epochs=30, batch_size=32)


Epoch 1/30
749/749 ━━━━━━━━━━━━━━━━━━━━ 130s 162ms/step - accuracy: 0.0188 - loss: 6.1979 - val_accuracy: 0.1979 - val_loss: 3.1786
Epoch 2/30
749/749 ━━━━━━━━━━━━━━━━━━━━ 114s 152ms/step - accuracy: 0.2707 - loss: 2.7950 - val_accuracy: 0.4943 - val_loss: 1.6682
Epoch 3/30
749/749 ━━━━━━━━━━━━━━━━━━━━ 112s 150ms/step - accuracy: 0.5276 - loss: 1.5804 - val_accuracy: 0.6712 - val_loss: 1.0441
Epoch 4/30
749/749 ━━━━━━━━━━━━━━━━━━━━ 115s 154ms/step - accuracy: 0.6663 - loss: 1.0720 - val_accuracy: 0.7710 - val_loss: 0.7077
Epoch 5/30
749/749 ━━━━━━━━━━━━━━━━━━━━ 114s 152ms/step - accuracy: 0.7498 - loss: 0.7821 - val_accuracy: 0.8181 - val_loss: 0.5525
Epoch 6/30
749/749 ━━━━━━━━━━━━━━━━━━━━ 114s 152ms/step - accuracy: 0.7956 - loss: 0.6311 - val_accuracy: 0.8570 - val_loss: 0.4653
Epoch 7/30
749/749 ━━━━━━━━━━━━━━━━━━━━ 114s 152ms/step - accuracy: 0.8334 - loss: 0.5067 - val_accuracy: 0.8699 - val_loss: 0.4057
Epoch 8/30
749/749 ━━━━━━━━━━━━━━━━━━━━ 117s 156ms/step - accuracy: 0.8479 -

KeyboardInterrupt: 

## Kết quả chỉ training với 10 epoch mà val_accuracy = 0.9041  (90%), có thể tốt hơn một số mô hình lớn hiện nay bởi vì họ sử dụng quá nhiều từ vựng, kết quả không tệ nhưng có thể dễ bị overtting... cần thử nghiệm thực tế nhiều hơn